# 모형의 선택
* 최적의 모형을 선택하기 위해 적당한 하이퍼파라미터의 설정이 중요하다.
* 여러가지의 후보 모형에서 효율적으로 최선의 모형을 선택하는 기법을 다룬다.
    * 완전탐색 기법 
    * 랜덤탐색 기법

### 완전탐색 기법
사이킷런의 GridSearchCV를 사용한다. 하이퍼파라미터의 **모든 조합**으로 모형을 훈련하여 최적의 하이퍼파라미터를 확인할 수 있다.

In [1]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression(solver='liblinear', max_iter=500)

하이퍼파라미터의 후보값을 설정한다.   
penalty 방법과 규제 변수인 C에 적당한 후보값을 설정한다.

In [2]:
# 페널티(penalty) 하이퍼파라미터 값의 후보를 만듭니다.
penalty = ['l1', 'l2']

# 규제 하이퍼파라미터 값의 후보 범위를 만듭니다.
C = [0.01, 0.1, 1, 10, 100]

# 하이퍼파라미터 후보 딕셔너리를 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)

* 사이킷런의 그리드 서치 기능은 모든 조합에 대해 모형을 훈련한다.  
* 최고의 성능을 내는 모형을 최종 모형으로 선택한다.  
* K-fold cross validation기법을 사용한다.
### GridSearchCV
* verbose =0 이면 아무것도 출력하지 않고 1~3 값을 주면 자세한 내용이 출력된다.
* n_jobs = -1 로 설정하면 모든 CPU 코어를 사용하여 병렬로 계산한다.

In [3]:
# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0)

# 그리드 서치를 수행합니다.
best_model = gridsearch.fit(features, target)

* 모든 조합에 대해 모형을 훈련하므로 시간이 많이 소요될 수 있다.
* 위 설정에서 패널티는 2개, C 규제변수의 값은 5개, fold 수는 5개  
* 총 2x5x5 = 50개의 모형중 최선의 모형을 선택한다.
* GridSearchCV가 완료되면 최선의 모형을 만드는 하이퍼파라미터를 확인할 수 있다.

In [4]:
# 최선의 하이퍼파라미터를 확인합니다.
print('가장 좋은 페널티:', best_model.best_estimator_.get_params()['penalty'])
print('가장 좋은 C 값:', best_model.best_estimator_.get_params()['C'])

가장 좋은 페널티: l1
가장 좋은 C 값: 10


In [5]:
# 최적의 하이퍼파라미터를 적용한 모형의 성능을 확인한다.
best_model.score(features, target)

0.98

### 랜덤탐색 기법
사이킷런의 RandomizedSearchCV을 사용한다. 완전탐색 기법보다는 최적 모형을 선택하는데 드는 계산 비용을 줄인다. 랜덤한 하이퍼파라미터의 조합으로 조사한다.

In [6]:
# 라이브러리를 임포트합니다.
from scipy.stats import uniform
from sklearn import linear_model, datasets
from sklearn.model_selection import RandomizedSearchCV

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 로지스틱 회귀 모델을 만듭니다.
logistic = linear_model.LogisticRegression(solver='liblinear')

하이퍼파라미터의 후보값을 설정한다.  
penalty 방법과 규제 변수인 C에 적당한 후보값을 설정한다.

### RandomizedSearchCV
RandomizedSearchCV에서 분포를 지정하면 이 분포에서 중복을 허용하지 않도록 하이퍼파라미터의 값을 랜덤하게 샘플링한다. 

In [7]:
# 페널티 하이퍼파라미터 후보를 만듭니다. penalty hyperparameter values
penalty = ['l1', 'l2']

# 규제 하이퍼파라미터 값의 후보를 위한 분포를 만듭니다.
C = uniform(loc=0, scale=100) # 0~100 사이의 균등분포에서 랜덤하게 샘플링한다.

# 하이퍼파라미터 옵션을 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)

# 랜덤 서치 객체를 만듭니다.
randomizedsearch = RandomizedSearchCV(
    logistic, hyperparameters, random_state=1, n_iter=100, cv=5, verbose=0,
    n_jobs=-1)

# 랜덤 서치를 수행합니다.
best_model = randomizedsearch.fit(features, target)

GridSearchCV에서와 마찬가지로 최적 모형의 하이퍼파라미터를 확인할 수 있다.

In [8]:
# 최선의 하이퍼파라미터를 확인합니다.
print('가장 좋은 페널티:', best_model.best_estimator_.get_params()['penalty'])
print('가장 좋은 C 값:', best_model.best_estimator_.get_params()['C'])

가장 좋은 페널티: l2
가장 좋은 C 값: 93.25573593386588


In [9]:
# 최적의 하이퍼파라미터를 적용한 모형의 성능을 확인한다.
best_model.score(features, target)

0.98

### 여러 학습 알고리즘에서 최선의 모델 선택하기

다양한 학습 알고리즘과 각각의 하이퍼파라미터를 탐색하여 최선의 모형을 선택한다.

In [11]:
# 라이브러리를 임포트합니다.
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# 랜덤 시드를 설정합니다.
np.random.seed(0)

# 데이터를 로드합니다.
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 파이프라인을 만듭니다.
pipe = Pipeline([("classifier", RandomForestClassifier())])

로지스틱 회귀 모형과 랜덤포레스트 두 개의 학습 알고리즘을 학습 탐색에 포함 시킨다. 
딕셔너리를 사용하여 학습 알고리즘과 탐색하고자하는 하이퍼파라미터를 지정한다.

In [14]:
# 후보 학습 알고리즘과 하이퍼파라미터로 딕셔너리를 만듭니다.
search_space = [{"classifier": [LogisticRegression(solver='liblinear', max_iter=500)],
                 "classifier__penalty": ['l1', 'l2'],
                 "classifier__C": np.logspace(0, 4, 10)},
                {"classifier": [RandomForestClassifier()],
                 "classifier__n_estimators": [10, 100, 1000],
                 "classifier__max_features": [1, 2, 3]}]

In [15]:
# 그리드 서치 객체를 만듭니다.
gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=0)

# 그리드 서치를 수행합니다.
best_model = gridsearch.fit(features, target)

In [16]:
# 최선의 모델을 확인합니다.
best_model.best_estimator_.get_params()["classifier"]

LogisticRegression(C=7.742636826811269, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)